In [1]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [2]:
model_path = "bert-driver4"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

labels = ["Products", "Innovation", "Workplace", "Governance", "Citizenship", "Leadership", "Performance"]

In [3]:
def predict_labels(text, threshold=0.5):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probs = torch.sigmoid(logits).cpu().numpy()[0]
    preds = (probs >= threshold).astype(int)
    return probs, preds

In [4]:
text = "Descalzi ha parlato alle famiglie dei dipendenti per augurare un buon anno"
probs, preds = predict_labels(text)
drivers = []
for i in range(len(labels)):
    if preds[i] == 1:
        drivers.append(labels[i])
drivers

['Workplace', 'Leadership']